In [19]:
import zipfile
import os
import glob

save_path = "../model/prediction/unSegmented_auto_generate_mask_nrrd/VNet_CTM"
subdir_pat = os.path.join(save_path,'*')
subdir_list = glob.glob(subdir_pat)

zip_dir = save_path+'_zip'
if not os.path.isdir(zip_dir):
    os.makedirs(zip_dir)

for sample_name in os.listdir(save_path):
#     print("%s compressing ..."%sample_name)
    subdir = os.path.join(save_path,sample_name)
    if not os.path.isdir( subdir ):
        continue

    # saved zip-file-name
    zip_filepath = os.path.join(zip_dir,sample_name)+".zip"
    
    # create zip object
    compressPathName = zip_filepath
    zip_file = zipfile.ZipFile(compressPathName, 'w', zipfile.ZIP_DEFLATED)
    # write
    for file in os.listdir(subdir):
        file_path = os.path.join(subdir,file)    # 会返回压缩包内所有文件名的列表。
        zip_file.write(file_path, file) # （2）将文件写入zip压缩文件——正常压缩，不出现多层目录

    # close
    zip_file.close()
    print("%s compression complete!"%sample_name)
    
print('compression complete!')


1204979 compressing ...
1204979 compression complete!
b1261721 compressing ...
b1261721 compression complete!
1302098 compressing ...
1302098 compression complete!
1336587-dingzi compressing ...
1336587-dingzi compression complete!
b1532717 compressing ...
b1532717 compression complete!
B1296091 compressing ...
B1296091 compression complete!
B1300089 compressing ...
B1300089 compression complete!
1478592 compressing ...
1478592 compression complete!
1291947 compressing ...
1291947 compression complete!
b1232165 compressing ...
b1232165 compression complete!
B1411961 compressing ...
B1411961 compression complete!
1204336 compressing ...
1204336 compression complete!
b1330821 compressing ...
b1330821 compression complete!
b1203609 compressing ...
b1203609 compression complete!
B1674780-1 compressing ...
B1674780-1 compression complete!
1510907 compressing ...
1510907 compression complete!
1740935 compressing ...
1740935 compression complete!
B1455577 compressing ...
B1455577 compression 